* Tracking: Theo dõi vị trí các đối tượng qua cá khung hình liên tiếp
* Counting: Đếm số lượng các đối tượng trong ảnh
* Open vocabulary: nhận điện các đối tượng không nằm trong danh sách label cố định

# Import

In [1]:
from collections import defaultdict
import cv2
import numpy as np
from ultralytics import YOLO

In [2]:
model = YOLO('yolo11l.pt')

video_path = 'samples/vietnam.mp4'
cap = cv2.VideoCapture(video_path)

100%|██████████| 49.0M/49.0M [00:01<00:00, 26.9MB/s]


In [7]:
# Get video properties
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Create VideoWriter object
video_name = video_path.split('/')[-1]
output_path = f'./run/{video_name.split('.')[0]}_tracked.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

- Khởi tạo lịch sử theo dõi và vòng lặp xử lý frames: Chúng ta sẽ sử dụng một defaultdict để lưu trữ lịch sử theo dõi các đối tượng và thực hiện vòng lặp qua từng frame của video để thực hiện việc theo dõi.


In [8]:
from collections import deque

track_history = defaultdict(lambda: deque())

while cap.isOpened():
    success, frame = cap.read()
    
    if not success:
        break
    
    # Run YOLO11 tracking on the frame, persisting tracks between frames
    results = model.track(frame, persist=True, show=False)
    
    # Get the boxes and track IDs (with error handling)
    boxes = results[0].boxes.xywh.cpu()
    try:
        track_ids = results[0].boxes.id
        if track_ids is not None:
            track_ids = track_ids.int().cpu().tolist()
        else:
            track_ids = [] # No tracks found in this frame
    except AttributeError:
        track_ids = [] # Handle case where tracking fails
    
    # Visualize the results on the frame    
    annotated_frame = results[0].plot()
    
    # Plot the tracks only if we have valid tracking data
    if track_ids:
        for box, track_id in zip(boxes, track_ids):
            x, y, w, h = box
            track = track_history[track_id]
            track.append((float(x), float(y))) # x, y center point
            
            if len(track) > 120: # retain 30 tracks for 30 frames
                track.popleft()
                
            # Draw the tracking lines
            points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
            cv2.polylines(
                annotated_frame,
                [points],
                isClosed=False,
                color=(230, 230, 230),
                thickness=4,
            )
    # Write the frame to output video
    out.write(annotated_frame)
    
cap.release()
out.release()
print(f'Video has been saved to {output_path}')
    

Video has been saved to ./run/vietnam_tracked.mp4
